In [ ]:
from utils.utils import *
from utils.utils_nn import *
from utils.Rnn_model import Rnn_model

import numpy as np
np.random.seed(seed())

import tensorflow as tf

import os

from tensorflow.contrib import rnn
from tensorflow.contrib.tensorboard.plugins import projector  # embeddings visualizer
# from tensorflow.python.framework import ops  # for custom actiavation function definition

import random
random.seed(seed())

# import matplotlib.pyplot as plt

import argparse

import itertools

In [ ]:
kwargs_neural_data_init = \
    {'mk_chars': True, 
               'model': 'neural', 
               'char_filter': 100, 'allowed_chars': None, 
               'mk_ngrams': False, 'ngram_width': 5, 
               'ngram_filter': 10, 'allowed_ngrams': None, 
               'keep_infreq_labels': False, 'label_count_thresh': 10, 
               'valid_ratio': 0.25, 
               'scale_func': unscale, 'to_permute': True, }

x_feed_train, y_feed_train, x_feed_val, y_feed_val,\
    char_int, char_int_inv, label_int, label_int_inv, \
    statistics_dict =\
    data_load_preprocess(**kwargs_neural_data_init)

In [ ]:
kwargs_simple_rnn = nice_dict({
    # log
    'log_dir': 'logdir/', 
    'del_log': True, 
    # preprocessing and data
    'top_k': 5, 
    'seed': seed(), 
    # learning hyper-params
    'learn_rate': 1E-2, 
    'dynamic_learn_rate': False, 
    'rnn_type': 'GRU',
    'bidirection': False, 
    'char_embed_dim': 4, 
    'one_hot': True,
    'hidden_state_size': 32, 
    'keep_prob': 0.7, 
    # noisy activation hyper params
    'activation_function': 'noisy_tanh',  # tf.tanh / 'noisy_tanh'
    'learn_p_delta_scale': False,   # noise scale param in noisy activation
    'noise_act_alpha': 1.15,  # mixing in the linear activation
    'noise_act_half_normal': False,
    # regularization constants
    'l2_weight_reg': 1E-3, 
    'target_rep': True, 
    'target_rep_weight': 0.3, 
    # training settings
    'epochs': 20,
    'summary_step': 10, 
    'save_step': np.inf,
    'to_save': False, 
    'verbose_summary': False
})

if kwargs_simple_rnn.save_step == np.inf and \
    kwargs_simple_rnn.to_save: 
    kwargs_simple_rnn.save_step = kwargs_simple_rnn.epochs
    
kwargs_simple_rnn = {**kwargs_simple_rnn, 
                      **statistics_dict}

kwargs_simple_rnn = nice_dict({**kwargs_simple_rnn, 
                               **{'scale_func': kwargs_neural_data_init['scale_func'], 
                                  'keep_infreq_labels': kwargs_neural_data_init['keep_infreq_labels']}})

if kwargs_simple_rnn.del_log: remove_dir_content(kwargs_simple_rnn.log_dir)

In [ ]:
"""
Enable passing some keyword arguments from command line.
This does not affect the Jupyter notebook.
"""
# try:
parser = argparse.ArgumentParser()

parser.add_argument('--logdir', action='store', dest='logdir',
                    help='Specify the log directory path', 
                    type=str,
                    default=None)

parser.add_argument('--rnn_type', action='store', dest='rnn_type',
                    help='Specify RNN type ("GRU" or "LSTM")', 
                    type=str, 
                    default=None)

parser.add_argument('--bidir', action='store', dest='bidir',
                    help='Specify data feed direction '+\
                        '(False for forward-feed or True for bidirectional)', 
                    type=str, 
                    default=None)

#     results = parser.parse_args()
args, unknown = parser.parse_known_args()

if args.logdir is not None and isinstance(args.logdir, str):
    kwargs_simple_rnn.log_dir = str(args.logdir)
if args.rnn_type is not None and isinstance(args.rnn_type, str):
    kwargs_simple_rnn.rnn_type = str(args.rnn_type)
if args.bidir is not None and isinstance(args.bidir, str):
    kwargs_simple_rnn.bidirection = args.bidir in ['True', 'T']

# except:
#     pass
# print(kwargs_simple_lstm.log_dir, type(kwargs_simple_lstm.log_dir))
# print(kwargs_simple_lstm.rnn_type, type(kwargs_simple_lstm.rnn_type))
# print(kwargs_simple_lstm.bidirection, type(kwargs_simple_lstm.bidirection))

In [ ]:
look_at_some_examples = False
"""
Collect examples from training and validation sets,
group by label and print two examples for each 
(e.g. for each label, print 2 training and 2 validation examples).
This was done due to a suspicion raised by similar evaulation metrics on the training and test.
"""
if look_at_some_examples:
    label_to_text_val = {}  # collect validation examples
    for obs,label in zip(x_feed_val, y_feed_val):
        label_to_text_val.setdefault(label,[]).append(obs)

    label_to_text_train = {}  # collect training examples
    for obs,label in zip(x_feed_train, y_feed_train):
        label_to_text_train.setdefault(label,[]).append(obs)

    unique_keys = list(label_to_text_train.keys())
    unique_keys.sort()

    label_to_text_merge = {}  # collect both
    for key in unique_keys:
        label_to_text_merge[key] = {'training': label_to_text_train[key], 
                                    'validation': label_to_text_val[key]}

    for key in unique_keys:
        cur_dict = label_to_text_merge[key]
        print('Key:{}, training:'.format(key))
        print(''.join([char for char in cur_dict['training'][0] if char != '<pad-char>']))
        print(''.join([char for char in cur_dict['training'][1] if char != '<pad-char>']))
        print('validation:')
        print(''.join([char for char in cur_dict['validation'][0] if char != '<pad-char>']))
        print(''.join([char for char in cur_dict['validation'][1] if char != '<pad-char>']))    

In [ ]:
# returns np.arrays to feed into tf model
# training data
X_train, _, Y_train = index_transorm_xy(x=x_feed_train, 
                                        y=y_feed_train, 
                                        char_int=char_int, 
                                        label_int=label_int, 
                                        **kwargs_simple_rnn)

# validation data
X_val, _, Y_val = index_transorm_xy(x=x_feed_val, 
                                    y=y_feed_val, 
                                    char_int=char_int, 
                                    label_int=label_int, 
                                    **kwargs_simple_rnn)

# write a metadata file for embeddings visualizer and create path string
embed_vis_path = write_embeddings_metadata(log_dir=kwargs_simple_rnn.log_dir, 
                                           dictionary=char_int, 
                                           file_name='metadata.tsv')

In [ ]:
# keep_first_k_chars(input=X_train, k=2, 
#                        model='neural', 
#                        char_int=char_int)

In [ ]:
kwargs_feed_dict_train = {'x': X_train, 'y': Y_train}
kwargs_feed_dict_test = {'x': X_val, 'y': Y_val}

hparam_str = make_hparam_string(**kwargs_simple_rnn)

rnn_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
#                       **{**kwargs_simple_rnn, 
#                       **{'epochs': 100}}
                **kwargs_simple_rnn
                      )

rnn_model.train()
rnn_model.close_session()

In [ ]:
# load weights from file

# hparam_str = make_hparam_string(**kwargs_simple_rnn)

rnn_new = Rnn_model(
#                   hparam_str=hparam_str, 
#                 embed_vis_path=embed_vis_path, 
#                 feed_dict_train=kwargs_feed_dict_train, 
                feed_dict_test=kwargs_feed_dict_test, 
                **{**kwargs_simple_rnn, 
                   **{'epochs': 100}}
#                 **kwargs_simple_rnn
                 )

In [ ]:
rnn_new.restore(cp_path=os.path.join(kwargs_simple_rnn.log_dir, hparam_str))

In [ ]:
[accuracy, cost, recip_rank, top_k] = rnn_new.run_eval()
print('accuracy is {:.5f}'.format(accuracy))

In [ ]:
rnn_new.update_test_dict(kwargs_feed_dict_train)
[accuracy, cost, recip_rank, top_k] = rnn_new.run_eval()
print('accuracy is {:.5f}'.format(accuracy))

In [ ]:
# with tf.Session() as sess:
# lstm.p_delta_scale.eval(session=lstm.sess)
# lstm.p_delta_scale

In [ ]:
# lstm.restore(cp_path=os.path.join(lstm.log_dir, hparam_str))

In [ ]:
# lstm.sess.run(lstm.outputs, feed_dict=lstm.feed_dict_train_eval)

In [ ]:
# trying out a LOT of hyper-parameters configurations
kwargs_feed_dict_train = {'x': X_train, 'y': Y_train}
kwargs_feed_dict_test = {'x': X_val, 'y': Y_val}

rnn_models = {}
for keep_prob in [0.7]:
#         for one_hot, char_embed_dim in [(True, 4)] + list(zip([False] * 1 , [4])):
    for hidden_state_size in [64, 128]:
        for l2_weight_reg in list(np.logspace(-3, -4, 2)):
            for (activation_function,  # all interaction / permutations for noisy activation
                 learn_p_delta_scale, 
                 noise_act_alpha, 
                 noise_act_half_normal) in [
                [tf.tanh, False, 1.15, False]] +\
                [[*j] for j in\
                list(itertools.product(['noisy_tanh'] ,
                               [True, False], 
                               [1.15, 0.9], 
                               [True, False]))]:
                    
                # collect new hyperparameters as args
                current_kw_simple_rnn = {
                    **kwargs_simple_rnn, 
                    **{'keep_prob': keep_prob, 
                       'hidden_state_size': hidden_state_size, 
                       'l2_weight_reg': l2_weight_reg, 
                       'activation_function': activation_function,
                       'learn_p_delta_scale': learn_p_delta_scale,
                       'noise_act_alpha': noise_act_alpha,
                       'noise_act_half_normal': noise_act_half_normal
    #                        'bidirection': bidirection, 
    #                        'target_rep': target_rep
                      }}
                hparam_str = make_hparam_string(**current_kw_simple_rnn)
                print(hparam_str)
                var = 'rnn_{}'.format(hparam_str)
                rnn_models[var] = rnn_model(feed_dict_train=kwargs_feed_dict_train, 
                                              feed_dict_test=kwargs_feed_dict_test, 
                                              hparam_str=hparam_str, 
                                              embed_vis_path=embed_vis_path, 
                                              **current_kw_simple_rnn)
                rnn_models[var].train()
                rnn_models[var].close_session()

In [ ]:
# getting data directly from a tensorboard log dir
from tensorflow.python.summary import event_multiplexer
# specify path (for parent log dir)
log_parent_dir = './logdir_exper_4_3/'
ea = event_multiplexer.EventMultiplexer().AddRunsFromDirectory(log_parent_dir)
ea.Reload()  # load

child_dir = next(os.walk(log_parent_dir))[1]
print(ea.Scalars(child_dir[0], 'accuracy/accuracy_test'))  # specify run, scalar_name

In [ ]:
# http://stackoverflow.com/questions/39921607/tensorflow-how-to-make-a-custom-activation-function-with-only-python

# def custom_tanh(x):  # spiky
#     return np.tanh(x)
# #     return x ** 2
# np_custom_tanh = np.vectorize(custom_tanh)  # np_spiky = np.vectorize(spiky)


# def d_custom_tanh(x):  # d_spiky
#     return 1 - np.tanh(x) ** 2
# #     return 2 * x
# np_d_custom_tanh = np.vectorize(d_custom_tanh)  # np_d_spiky = np.vectorize(d_spiky)



# def tf_d_custom_tanh(x,name=None, stateful=False):  # tf_d_spiky
#     """
#     Converting a Numpy function to a Tensorflow function.
#     tf.py_func acts on lists of tensors and returns a list of tensors.
#     stateful, if the same input might produce a different outputs (stochastic).
#     """
#     with ops.name_scope(name, 
#                         default_name='d_custom_act', 
#                         values=[x]) as name:
#         result = tf.py_func(lambda x: np_d_custom_tanh(x).astype(np.float32),
#                         [x],
#                         [tf.float32],
#                         name=name,
#                         stateful=stateful)
#         return result[0]


# def py_func(func, input, type_out, stateful=True, name=None, grad=None):
#     """
#     Modify the tf.py_func function to make it define the gradient at the same time
#     """
#     # Need to generate a unique name to avoid duplicates:
#     rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))

#     tf.RegisterGradient(rnd_name)(grad)  # see _MySquareGrad for grad example
#     g = tf.get_default_graph()
#     with g.gradient_override_map({"PyFunc": rnd_name}):
#         return tf.py_func(func, input, type_out, stateful=stateful, name=name)


# def custom_tanh_grad(op, grad):  # spikygrad
#     """
#     py_func requires a function of a particular form, 
#     taking an operation and a 'gradient' and returning the computed gradient.
#     """
#     x = op.inputs[0]
#     n_gr = tf_d_custom_tanh(x)
# #     print('op is:', op)
#     print('grad is:', grad)
#     print('x is:', x)
#     print('n_gr is:', n_gr)
#     return grad * n_gr
# #     return [g * tf_d_custom_tanh(inp) for g, inp in zip(grad ,x)]


# def tf_custom_tanh(x, name=None):  # tf_spiky

#     with ops.name_scope(name, 
#                         default_name='custom_act', 
#                         values=[x]) as name:
#         result = py_func(lambda x: np_custom_tanh(x).astype(np.float32),
#                         [x],
#                         [tf.float32],
#                         name=name,
#                         grad=custom_tanh_grad)  # <-- here's the call to the gradient
#         return result[0]

In [ ]:
# # based Noisy Activation Functions paper
# # https://arxiv.org/abs/1603.00391
# # https://github.com/caglar/noisy_units/blob/master/codes/tf/nunits.py

# def lin_sigmoid(x):
#     """
#     First-order Taylor expansion around zero of the sigmoid function
#     """
#     return 0.25 * x + 0.5


# def hard_sigmoid(x):
#     """
#     Hard saturating sigmoid function, with clipping applied
#     """
#     return tf.minimum(tf.maximum(lin_sigmoid(x), 0.0), 1.0)


# def noise_hard_tanh_sat(x, use_noise, stddev=0.25):
#     """
#     Noisy Hard Tanh Units at Saturation: NANIS as proposed in the paper
#     https://arxiv.org/abs/1603.00391
#     Arguments:
#         x: input tensor variable.
#         use_noise: bool, whether to add noise or not (useful for test time)
#         c: float, standard deviation of the noise
#     """
#     threshold = 1.001  # point where the unit is saturated, in abs terms
#     def noise_func() :return tf.random_normal(tf.shape(x), mean=0.0, stddev=1.0, dtype=tf.float32)
#     def zero_func (): return tf.zeros(tf.shape(x), dtype=tf.float32, name=None)
#     noise = tf.cond(use_noise,noise_func,zero_func)  # add noise or zeroes
    
#     test = tf.cast(tf.greater(tf.abs(x) , threshold), tf.float32)
#     res = test * hard_tanh(x + stddev * noise) + (1.0 - test) * hard_tanh(x)
#     return res